In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments

from datasets import Dataset, DatasetDict, concatenate_datasets, load_dataset

2025-03-25 17:46:07.301500: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-25 17:46:07.310209: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742913967.319364   26281 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742913967.322282   26281 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-25 17:46:07.333027: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
from unsloth import FastLanguageModel
import torch

max_seq_length = (2048*2) # Choose any! We auto support RoPE Scaling internally!
dtype = torch.bfloat16 # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = False # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/llama-2-13b-bnb-4bit",
    "unsloth/codellama-34b-bnb-4bit",
    "unsloth/tinyllama-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit", # New Google 6 trillion tokens model 2.5x faster!
    "unsloth/gemma-2b-bnb-4bit",
] # More models at https://huggingface.co/unsloth



🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-bnb-4bit", # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B Unsloth-SmolLM2-360M-Lora
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    trust_remote_code=True,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

Unsloth: WARNING `trust_remote_code` is True.
Are you certain you want to do remote code execution?
==((====))==  Unsloth 2025.2.15: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    GPU: NVIDIA GeForce RTX 3090. Max memory: 23.584 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu118. CUDA: 8.6. CUDA Toolkit: 11.8. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,
    lora_dropout = 0.2, # Supports any, but = 0 is optimized
    bias = "all",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config ={"bits": 4}, # And LoftQ
    init_lora_weights="loftq",
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.2.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth: bias = `none` is supported for fast patching. You are using bias = all.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.2.15 patched 28 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [5]:
system_prompt_text = ''' 
Sen bir tarım uzmanısın. Kullanıcının yalnızca tarım, bitki hastalıkları, tarımsal ilaçlar, tarım ekipmanları, sulama sistemleri, gübreleme teknikleri ve çiftçilikle ilgili konularda sorularını teknik ve detaylı bir şekilde cevapla.  

Başka bir konu sorulduğunda, yalnızca tarımla ilgili konular hakkında yardımcı olabileceğini belirt.  

Ziraat ile ilgili anahtar kelimeler:  
- Bitki hastalıkları, zararlılar, fungusitler, herbisitler, insektisitler  
- Organik tarım, gübreleme, toprak analizi, sürdürülebilir tarım  
- Sulama sistemleri, damla sulama, pivot sulama, yağmurlama  
- Traktörler, biçerdöverler, ekim makineleri, tarımsal mekanizasyon  
- Meyvecilik, sebzecilik, tahıl üretimi, seracılık  
- Tarımsal biyoteknoloji, genetiği değiştirilmiş organizmalar (GDO)  
- İklim değişikliği ve tarıma etkileri, kuraklık yönetimi  

Eğer kullanıcı tarım dışı bir konu sorarsa, "Ben bir tarım uzmanıyım ve yalnızca tarımla ilgili konularda yardımcı olabilirim." şeklinde yanıt ver.
'''

In [6]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.2",
    #mapping={"role": "from", "content": "value", "user": "human", "assistant": "gpt"},
)


def questions2gptFormat(rows):
    conversations = []
    
    # Sistem promptunu ekleyelim
    system_prompt = [
        {"from": "system", "value": system_prompt_text}
    ]

    def format_row(row):
        # Kullanıcı ve asistan mesajlarını ekleme
        conversations.append([{"from": "system", "value": system_prompt_text}, {"from": "human", "value": row[0]}, {"from": "gpt", "value": row[1]}])
                
    for i in zip(rows["instruction"], rows["output"]):
        format_row(i)

    return {"conversations": conversations}

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

In [7]:

from datasets import load_dataset
# 2. Tapaco Veri Setini Yükleyin

dataset = Dataset.from_csv("BKÜ Sınav Analizi - BKÜ Sınav Analizi.csv")
dataset_test = Dataset.from_csv("tarim_veriseti_test.csv").select(range(2))

In [8]:
dataset = dataset.filter(lambda x: x["GPT"]!=None and x["Soru"]!=None)
dataset = dataset.filter(lambda x: len(x["GPT"])<max_seq_length)

In [9]:
dataset

Dataset({
    features: ['Soru', 'Cevap', 'Doğru Cevap', 'GPT', 'inputs'],
    num_rows: 16368
})

In [10]:
dataset_test


Dataset({
    features: ['Soru', 'Cevap'],
    num_rows: 2
})

In [11]:
dataset = dataset.rename_columns({"Soru": "instruction", "GPT": "output"})
dataset_test = dataset_test.rename_columns({"Soru": "instruction", "Cevap": "output"})

In [12]:
from datasets import Dataset
import numpy as np

# None içeren satırları temizleyen fonksiyon
def remove_none_rows(example):
    return example["instruction"] is not None and example["output"] is not None

# None değerleri içeren satırları filtrele
dataset = dataset.filter(remove_none_rows)

In [13]:
dataset

Dataset({
    features: ['instruction', 'Cevap', 'Doğru Cevap', 'output', 'inputs'],
    num_rows: 16368
})

In [14]:
dataset = dataset.map(questions2gptFormat, batched = True, batch_size=10000).remove_columns(["instruction",'Cevap', 'Doğru Cevap', 'inputs', "output"])

In [15]:
dataset[0]

{'conversations': [{'from': 'system',
   'value': ' \nSen bir tarım uzmanısın. Kullanıcının yalnızca tarım, bitki hastalıkları, tarımsal ilaçlar, tarım ekipmanları, sulama sistemleri, gübreleme teknikleri ve çiftçilikle ilgili konularda sorularını teknik ve detaylı bir şekilde cevapla.  \n\nBaşka bir konu sorulduğunda, yalnızca tarımla ilgili konular hakkında yardımcı olabileceğini belirt.  \n\nZiraat ile ilgili anahtar kelimeler:  \n- Bitki hastalıkları, zararlılar, fungusitler, herbisitler, insektisitler  \n- Organik tarım, gübreleme, toprak analizi, sürdürülebilir tarım  \n- Sulama sistemleri, damla sulama, pivot sulama, yağmurlama  \n- Traktörler, biçerdöverler, ekim makineleri, tarımsal mekanizasyon  \n- Meyvecilik, sebzecilik, tahıl üretimi, seracılık  \n- Tarımsal biyoteknoloji, genetiği değiştirilmiş organizmalar (GDO)  \n- İklim değişikliği ve tarıma etkileri, kuraklık yönetimi  \n\nEğer kullanıcı tarım dışı bir konu sorarsa, "Ben bir tarım uzmanıyım ve yalnızca tarımla ilgili

In [16]:
dataset

Dataset({
    features: ['conversations'],
    num_rows: 16368
})

In [17]:
dataset = dataset.shuffle(seed=42)
dataset = dataset.shuffle(seed=41)
dataset = dataset.shuffle(seed=40)
dataset = dataset.shuffle(seed=39)

In [18]:
from unsloth.chat_templates import standardize_sharegpt
dataset = standardize_sharegpt(dataset)
dataset = dataset.map(formatting_prompts_func, batched = True,)

In [19]:
dataset

Dataset({
    features: ['conversations', 'text'],
    num_rows: 16368
})

In [20]:
print(dataset[5]["text"])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

 
Sen bir tarım uzmanısın. Kullanıcının yalnızca tarım, bitki hastalıkları, tarımsal ilaçlar, tarım ekipmanları, sulama sistemleri, gübreleme teknikleri ve çiftçilikle ilgili konularda sorularını teknik ve detaylı bir şekilde cevapla.  

Başka bir konu sorulduğunda, yalnızca tarımla ilgili konular hakkında yardımcı olabileceğini belirt.  

Ziraat ile ilgili anahtar kelimeler:  
- Bitki hastalıkları, zararlılar, fungusitler, herbisitler, insektisitler  
- Organik tarım, gübreleme, toprak analizi, sürdürülebilir tarım  
- Sulama sistemleri, damla sulama, pivot sulama, yağmurlama  
- Traktörler, biçerdöverler, ekim makineleri, tarımsal mekanizasyon  
- Meyvecilik, sebzecilik, tahıl üretimi, seracılık  
- Tarımsal biyoteknoloji, genetiği değiştirilmiş organizmalar (GDO)  
- İklim değişikliği ve tarıma etkileri, kuraklık yönetimi  

Eğer kullanıcı tarım dışı bir konu 

In [21]:
print(dataset[5]["conversations"])

[{'content': ' \nSen bir tarım uzmanısın. Kullanıcının yalnızca tarım, bitki hastalıkları, tarımsal ilaçlar, tarım ekipmanları, sulama sistemleri, gübreleme teknikleri ve çiftçilikle ilgili konularda sorularını teknik ve detaylı bir şekilde cevapla.  \n\nBaşka bir konu sorulduğunda, yalnızca tarımla ilgili konular hakkında yardımcı olabileceğini belirt.  \n\nZiraat ile ilgili anahtar kelimeler:  \n- Bitki hastalıkları, zararlılar, fungusitler, herbisitler, insektisitler  \n- Organik tarım, gübreleme, toprak analizi, sürdürülebilir tarım  \n- Sulama sistemleri, damla sulama, pivot sulama, yağmurlama  \n- Traktörler, biçerdöverler, ekim makineleri, tarımsal mekanizasyon  \n- Meyvecilik, sebzecilik, tahıl üretimi, seracılık  \n- Tarımsal biyoteknoloji, genetiği değiştirilmiş organizmalar (GDO)  \n- İklim değişikliği ve tarıma etkileri, kuraklık yönetimi  \n\nEğer kullanıcı tarım dışı bir konu sorarsa, "Ben bir tarım uzmanıyım ve yalnızca tarımla ilgili konularda yardımcı olabilirim." şekl

In [22]:
train_dataset, eval_dataset = dataset.train_test_split(test_size=0.2, seed=42).values()


In [23]:
print(eval_dataset[5])

{'conversations': [{'content': ' \nSen bir tarım uzmanısın. Kullanıcının yalnızca tarım, bitki hastalıkları, tarımsal ilaçlar, tarım ekipmanları, sulama sistemleri, gübreleme teknikleri ve çiftçilikle ilgili konularda sorularını teknik ve detaylı bir şekilde cevapla.  \n\nBaşka bir konu sorulduğunda, yalnızca tarımla ilgili konular hakkında yardımcı olabileceğini belirt.  \n\nZiraat ile ilgili anahtar kelimeler:  \n- Bitki hastalıkları, zararlılar, fungusitler, herbisitler, insektisitler  \n- Organik tarım, gübreleme, toprak analizi, sürdürülebilir tarım  \n- Sulama sistemleri, damla sulama, pivot sulama, yağmurlama  \n- Traktörler, biçerdöverler, ekim makineleri, tarımsal mekanizasyon  \n- Meyvecilik, sebzecilik, tahıl üretimi, seracılık  \n- Tarımsal biyoteknoloji, genetiği değiştirilmiş organizmalar (GDO)  \n- İklim değişikliği ve tarıma etkileri, kuraklık yönetimi  \n\nEğer kullanıcı tarım dışı bir konu sorarsa, "Ben bir tarım uzmanıyım ve yalnızca tarımla ilgili konularda yardımcı

In [24]:
#FastLanguageModel.for_inference(model)

In [25]:
""" 
example = eval_dataset[5]
    
input_text = example["instruction"]
reference_text = example["output"]
inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
output_ids = model.generate(**inputs, max_new_tokens=max_seq_length)
decoded_output = tokenizer.decode(output_ids[0], skip_special_tokens=True) """

' \nexample = eval_dataset[5]\n    \ninput_text = example["instruction"]\nreference_text = example["output"]\ninputs = tokenizer(input_text, return_tensors="pt").to(model.device)\noutput_ids = model.generate(**inputs, max_new_tokens=max_seq_length)\ndecoded_output = tokenizer.decode(output_ids[0], skip_special_tokens=True) '

In [26]:
import wandb

wandb.init(project="Basic LLM Train", name="Llama 3B v3.2 r32 qlora", resume="allow",id="6j747x6c" ) #id="a7zeymst",id="ecibz7e4" id="dbaxrwf4"
wandb.watch(model, log="all")


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: h-osmankarabulut. Use `wandb login --relogin` to force relogin


In [28]:
import wandb
import evaluate
from transformers import TrainerCallback

class WandbRougeCallback(TrainerCallback):
    def __init__(self, test_dataset):
        self.rouge = evaluate.load("rouge")
        self.test_dataset = test_dataset
        
    
    def on_evaluate(self, args, state, control, **kwargs):
        model = kwargs.get("model", None)
        tokenizer = kwargs.get("tokenizer", None)
        FastLanguageModel.for_inference(model)
        if model is None or tokenizer is None:
            return
        
        predictions = []
        references = []
        
        for example in self.test_dataset:
            input_text = example["instruction"]
            reference_text = example["output"]
            inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
            output_ids = model.generate(**inputs, max_new_tokens=max_seq_length)
            decoded_output = tokenizer.decode(output_ids[0], skip_special_tokens=True)
            
            predictions.append(decoded_output)
            references.append(reference_text)
        
        scores = self.rouge.compute(predictions=predictions, references=references)
        
        wandb.log({
            "ROUGE-1": scores["rouge1"],
            "ROUGE-2": scores["rouge2"],
            "ROUGE-L": scores["rougeL"]
        })
    
    def on_train_end(self, args, state, control, **kwargs):
        wandb.finish()

class WandbBleuCallback(TrainerCallback):
    def __init__(self, test_dataset):
        self.bleu = evaluate.load("bleu")
        self.test_dataset = test_dataset
        
    
    def on_evaluate(self, args, state, control, **kwargs):
        model = kwargs.get("model", None)
        tokenizer = kwargs.get("tokenizer", None)
        FastLanguageModel.for_inference(model)
        if model is None or tokenizer is None:
            return
        
        predictions = []
        references = []
        
        for example in self.test_dataset:
            input_text = example["instruction"]
            reference_text = example["output"]
            inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
            output_ids = model.generate(**inputs, max_new_tokens=max_seq_length)
            decoded_output = tokenizer.decode(output_ids[0], skip_special_tokens=True)
            
            predictions.append(decoded_output)
            references.append(reference_text)
        
        scores = self.bleu.compute(predictions=predictions, references=[[ref] for ref in references])
        
        wandb.log({
            "BLEU": scores["bleu"]
        })
    
    def on_train_end(self, args, state, control, **kwargs):
        wandb.finish()

class WandbMeteorCallback(TrainerCallback):
    def __init__(self, test_dataset):
        self.meteor = evaluate.load("meteor")
        self.test_dataset = test_dataset
        
    
    def on_evaluate(self, args, state, control, **kwargs):
        model = kwargs.get("model", None)
        tokenizer = kwargs.get("tokenizer", None)
        FastLanguageModel.for_inference(model)
        if model is None or tokenizer is None:
            return
        
        predictions = []
        references = []
        
        for example in self.test_dataset:
            input_text = example["instruction"]
            reference_text = example["output"]
            inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
            output_ids = model.generate(**inputs, max_new_tokens=max_seq_length)
            decoded_output = tokenizer.decode(output_ids[0], skip_special_tokens=True)
            
            predictions.append(decoded_output)
            references.append(reference_text)
        
        scores = self.meteor.compute(predictions=predictions, references=references)
        
        wandb.log({
            "METEOR": scores["meteor"]
        })
    
    def on_train_end(self, args, state, control, **kwargs):
        wandb.finish()

class WandbBertScoreCallback(TrainerCallback):
    def __init__(self, test_dataset):
        self.bertscore = evaluate.load("bertscore")
        self.test_dataset = test_dataset
        
    
    def on_evaluate(self, args, state, control, **kwargs):
        model = kwargs.get("model", None)
        tokenizer = kwargs.get("tokenizer", None)
        FastLanguageModel.for_inference(model)
        if model is None or tokenizer is None:
            return
        
        predictions = []
        references = []
        
        for example in self.test_dataset:
            input_text = example["instruction"]
            reference_text = example["output"]
            inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
            output_ids = model.generate(**inputs, max_new_tokens=max_seq_length)
            decoded_output = tokenizer.decode(output_ids[0], skip_special_tokens=True)
            
            predictions.append(decoded_output)
            references.append(reference_text)
        
        scores = self.bertscore.compute(predictions=predictions, references=references, lang="tr")
        
        wandb.log({
        "BERTScore Precision": np.mean(scores["precision"]),
        "BERTScore Recall": np.mean(scores["recall"]),
        "BERTScore F1": np.mean(scores["f1"])
    })
    
    def on_train_end(self, args, state, control, **kwargs):
        wandb.finish()


In [29]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 6,
    packing = False, # Can make training 5x faster for short sequences.
    #callbacks=[wandb_callback],
    args = TrainingArguments(
       
        gradient_accumulation_steps = 2,
        num_train_epochs=100,  
        per_device_train_batch_size=32,       # GPU başına batch boyutu
        per_device_eval_batch_size=32,       # GPU başına batch boyutu
        learning_rate = 0.001,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 300,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        eval_steps=1000,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "Unsloth-Llama-3B-v3.2-BKU-GPT-r32-qlora",
        report_to="wandb",                    # WandB veya diğer araçlara raporlama yok
        save_total_limit=2,                  # Sadece son iki checkpoint'i sakla
        save_steps=500,
        warmup_steps=2000,           # İlk 1000 adımda LR'yi yavaş yavaş artır
        
    ),
)

Tokenizing train dataset (num_proc=6):   0%|          | 0/13094 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=6):   0%|          | 0/13094 [00:00<?, ? examples/s]

Tokenizing eval dataset (num_proc=6):   0%|          | 0/3274 [00:00<?, ? examples/s]

Tokenizing eval dataset (num_proc=6):   0%|          | 0/3274 [00:00<?, ? examples/s]

In [30]:
trainer.add_callback(WandbRougeCallback(dataset_test))
trainer.add_callback(WandbBleuCallback(dataset_test))
trainer.add_callback(WandbMeteorCallback(dataset_test))
trainer.add_callback(WandbBertScoreCallback(dataset_test))

[nltk_data] Downloading package wordnet to /home/hosman/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/hosman/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/hosman/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [31]:
""" # Model ve tokenizer'ı eklediğinden emin ol
model.eval()
tokenizer.pad_token = tokenizer.eos_token

# Callback'leri test etmek için manuel çağırma
eval_callbacks = [
   # WandbRougeCallback(dataset_test),
    #WandbBleuCallback(dataset_test),
    #WandbMeteorCallback(dataset_test),
    WandbBertScoreCallback(dataset_test),
]

for i, callback in enumerate(eval_callbacks):
    print(i)
    callback.on_evaluate(None, None, None, model=model, tokenizer=tokenizer)

print("Evaluation metric test completed!")
 """

' # Model ve tokenizer\'ı eklediğinden emin ol\nmodel.eval()\ntokenizer.pad_token = tokenizer.eos_token\n\n# Callback\'leri test etmek için manuel çağırma\neval_callbacks = [\n   # WandbRougeCallback(dataset_test),\n    #WandbBleuCallback(dataset_test),\n    #WandbMeteorCallback(dataset_test),\n    WandbBertScoreCallback(dataset_test),\n]\n\nfor i, callback in enumerate(eval_callbacks):\n    print(i)\n    callback.on_evaluate(None, None, None, model=model, tokenizer=tokenizer)\n\nprint("Evaluation metric test completed!")\n '

In [32]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = f"<|start_header_id|>system<|end_header_id|>{system_prompt_text}<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Map:   0%|          | 0/13094 [00:00<?, ? examples/s]

Map:   0%|          | 0/3274 [00:00<?, ? examples/s]

In [33]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

'<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n \nSen bir tarım uzmanısın. Kullanıcının yalnızca tarım, bitki hastalıkları, tarımsal ilaçlar, tarım ekipmanları, sulama sistemleri, gübreleme teknikleri ve çiftçilikle ilgili konularda sorularını teknik ve detaylı bir şekilde cevapla.  \n\nBaşka bir konu sorulduğunda, yalnızca tarımla ilgili konular hakkında yardımcı olabileceğini belirt.  \n\nZiraat ile ilgili anahtar kelimeler:  \n- Bitki hastalıkları, zararlılar, fungusitler, herbisitler, insektisitler  \n- Organik tarım, gübreleme, toprak analizi, sürdürülebilir tarım  \n- Sulama sistemleri, damla sulama, pivot sulama, yağmurlama  \n- Traktörler, biçerdöverler, ekim makineleri, tarımsal mekanizasyon  \n- Meyvecilik, sebzecilik, tahıl üretimi, seracılık  \n- Tarımsal biyoteknoloji, genetiği değiştirilmiş organizmalar (GDO)  \n- İklim değişikliği ve tarıma etkileri, kuraklık yönetimi  \n

In [34]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

'                                                                                                                                                                                                                                                                                                                                              \n\nAyçiçeği, yağ üretimi ve hayvan yemi olarak kullanılır.<|eot_id|>'

In [ ]:
wandb.watch(model, log="all")
trainer_stats = trainer.train(resume_from_checkpoint=True)

/home/hosman/anaconda3/envs/torchEnv/lib/python3.11/site-packages/transformers/trainer.py:3420: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(os.path.join(checkpo

  0%|          | 0/20500 [00:00<?, ?it/s]

/home/hosman/anaconda3/envs/torchEnv/lib/python3.11/site-packages/transformers/trainer.py:3083: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint_rng_state = torch.lo

{'loss': 0.2259, 'grad_norm': 0.4956400692462921, 'learning_rate': 0.0008594594594594594, 'epoch': 22.44}


In [ ]:
model.save_pretrained("Unsloth-Llama-3B-v3.2-BKU-GPT-r32-qlora") # Local saving
tokenizer.save_pretrained("Unsloth-Llama-3B-v3.2-BKU-GPT-r32-qlora")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

INFERENCE

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = torch.bfloat16 # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/llama-2-13b-bnb-4bit",
    "unsloth/codellama-34b-bnb-4bit",
    "unsloth/tinyllama-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit", # New Google 6 trillion tokens model 2.5x faster!
    "unsloth/gemma-2b-bnb-4bit",
] # More models at https://huggingface.co/unsloth



In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Unsloth-Llama-3B-v3.2-BKU-GPT-r32-qlora/checkpoint-3300",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    trust_remote_code=True,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.2",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Ayçiçeği nedir?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 512, use_cache = True,
                         temperature = 1.5, min_p = 0.1)
decoded_outputs=tokenizer.batch_decode(outputs)

model_answer = decoded_outputs[0].replace("<|im_start|>user\n", "").replace("<|im_start|>assistant\n", "").replace("<|im_end|>", "").strip()
#model_answer = model_answer[len(question):]
model_answer

In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Ayçiçeği ekim zamanı ne zaman?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

In [ ]:
""" import pandas as pd
import torch
import random
from unsloth.chat_templates import get_chat_template
from difflib import SequenceMatcher
from tqdm import tqdm

# GPU kullanımı kontrol et
device = "cuda" if torch.cuda.is_available() else "cpu"

# Chat şablonunu Tokenizer'a uygula
tokenizer = get_chat_template(
    tokenizer,
    chat_template="chatml",
    mapping={
        "role": "from",
        "content": "value",
        "user": "human",
        "assistant": "gpt"
    },
    map_eos_token=True
)

# Modeli çıkarım (inference) için hazırla
FastLanguageModel.for_inference(model)
model.to(device)  # GPU'ya taşı

# CSV dosyasını oku
csv_file = "BKÜ Sınav Analizi.csv"
df = pd.read_csv(csv_file)

# Veriyi karıştır ve sadece %25'ini kullan
df = df.sample(frac=1, random_state=42)  # Karıştır
df = df.sample(frac=0.99, random_state=42)  # %25'ini seç

# Doğru tahminleri saymak için sayaç
correct_count = 0
total_questions = len(df)

# Benzerlik hesaplayan fonksiyon
def similarity(a, b):
    return SequenceMatcher(None, a.lower(), b.lower()).ratio()

# Soruları tek tek modele gönder ve doğruluğu ölç
for index, row in tqdm(df.iterrows(), total=len(df)):
    question = row["Soru"]
    correct_answer = row["GPT"]

    messages = [{"from": "human", "value": question}]

    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(device)

    outputs = model.generate(
        input_ids=inputs,
        max_new_tokens=512,
        use_cache=True
    )

    decoded_outputs = tokenizer.batch_decode(outputs)
    
    # Model çıktısını formatla
    model_answer = decoded_outputs[0].replace("<|im_start|>user\n", "").replace("<|im_start|>assistant\n", "").replace("<|im_end|>", "").strip()
    model_answer = model_answer[len(question):]

    # Benzerlik oranını hesapla
    match_ratio = similarity(str(model_answer), str(correct_answer))

    # %80'den büyükse doğru kabul et
    if match_ratio > 0.8:
        correct_count += 1

    #print(f"Soru: {question}")
    #print(f"Model Cevabı: {model_answer}")
    #print(f"Gerçek Cevap: {correct_answer}")
    #print(f"Benzerlik: %{match_ratio * 100:.2f}")
   # print("-" * 50)

# Doğruluk yüzdesini hesapla
accuracy = (correct_count / total_questions) * 100
print(f"Modelin doğruluk oranı: %{accuracy:.2f}")
 """